In [1]:
import os
import re
import pandas as pd
from pandas import Series,DataFrame
import numpy as np

In [2]:
alldata = []

In [3]:
# (using this as oppossed to native Pandas because txt doesn't injest easilly)
pdf_file = 'FormII.txt'
pdf = open(pdf_file, "r")

In [4]:
# puts the data from the pdf into a list, skipping new line characters
for line in pdf:
    alldata.append(line.strip())

In [5]:
alldata

['American Land Title Association',
 '',
 '',
 'ALTA Commitment',
 '',
 'Adopted 6-17-06',
 '',
 'CHICAGO TITLE INSURANCE COMPANY  800-943-1196',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'SCHEDULE A',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Loan No.:',
 '',
 'Title No.:  NC2',
 '',
 '1.  Effective date:  9th day of March, 2017',
 '',
 '',
 'Issue Date: 03/17/2017',
 '',
 'This Title Insurance Commitment is good for 60 days from the effective date. If you close a transaction based on',
 'this  Title  Insurance  Commitment  without  a  title  update,  Chicago  Title  Insurance  Company  will  not  issue  a  Title',
 'Insurance Policy.',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Policy',
 'Amount',
 '',
 'TBD',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '3.',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '2.',
 '',
 '',
 'Policy (or Policies) to be issued:',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 "(a)  Owner\xe2\x80\x99s Policy 2006

In [6]:
# makes a data frame out of the data 
raw_pdf = pd.DataFrame(alldata)

In [7]:
# finds the rows with blank data and fills them with Nan so they can be dropped, and drops them
raw_pdf = raw_pdf[raw_pdf != '']
raw_pdf.dropna(inplace=True)

In [8]:
# lets reindex so that we don't have huge jumps in the index for dropped rows
raw_pdf.set_index(np.arange(0,len(raw_pdf)),inplace = True)

In [9]:
# creating the list of headings to hunt for, hard-cording from the pdf, and using pandas tricks to get the last line.
headings = ['SCHEDULE A','LEGAL DESCRIPTION','REQUIREMENTS', 'EXCEPTIONS' , 'NOTE',(raw_pdf[0][raw_pdf.index[-1]])]

In [10]:
# create a dataframe to store the index locations for the starting line of each section
locations = [0]
df_locations = pd.DataFrame(locations)

In [11]:
# run each heading in our list through a search, and place those start index locations into a dataframe
for heading in headings:
    df_locations[heading] = raw_pdf[raw_pdf[0].str.match(heading)].index[0]

In [12]:
# some column labeling and cleanup, and sorting by order they appear in the pdf
df_locations.drop(0,axis=1,inplace=True)
df_locations = df_locations.rename({0:'start_line'}).transpose()
df_locations.sort_values(by='start_line',inplace=True)

In [13]:
# shifting to a numeric index so it is easy to iterate over rows in Dataframe
df_locations['sections'] = df_locations.index
df_locations.reset_index(drop=True, inplace=True)

In [14]:
# creating the column indicating the ending line for each section of the pdf
end = df_locations['start_line'][1:len(df_locations)].values
end = np.append(end,df_locations['start_line'][len(df_locations)-1:len(df_locations)].values)
df_locations['end_line'] = end

In [15]:
df_locations

,start_line,sections,end_line
0,4,SCHEDULE A,73
1,73,LEGAL DESCRIPTION,85
2,85,REQUIREMENTS,180
3,180,EXCEPTIONS,248
4,248,NOTE,250
5,250,"dimensions, distances or acreage shown thereon.",250


In [16]:
# now we can use "start_line" and "end_line" to define slices of the raw_pdf Dataframe and make section-specific chunks
# and output results to csv
for index, row in df_locations.iterrows():
    string = (pdf_file + "_" + "df" + "_" + str(row[1]))  
    print string
    string = raw_pdf[row[0]: row[2]]
    string.to_csv(path_or_buf=(pdf_file + "_" + "df" + "_" +str(row[1]) + ".csv"))

FormII.txt_df_SCHEDULE A
FormII.txt_df_LEGAL DESCRIPTION
FormII.txt_df_REQUIREMENTS
FormII.txt_df_EXCEPTIONS
FormII.txt_df_NOTE
FormII.txt_df_dimensions, distances or acreage shown thereon.


In [17]:
os.path.basename(os.getcwd())

'PDF_Study_Statestitle_052117'

In [18]:
os.getcwd()

'/Users/rondeepchahal/Downloads/StatesTitle/PDF_Study_Statestitle_052117'